In [13]:
from fetchYahooStockData import metadata_price_action, extended_stock_stats, basic_stats, current_sp500_symbols, parse_value, current_dow30_symbols
import numpy as np
import pandas as pd

In [51]:
symbol="AMD"
timeInterval = "1d"
longTimeInterval = "1wk"
timeRange = "5y"

In [52]:
metadataAndPrice = metadata_price_action(symbol, timeInterval, timeRange)
metadata = metadataAndPrice["metadata"]
metadata

https://query1.finance.yahoo.com/v8/finance/chart/AMD?symbol=AMD&region=US&lang=en-US&includePrePost=false&interval=1d&useYfid=true&range=5y&corsDomain=finance.yahoo.com&.tsrc=finance


{'currency': 'USD',
 'symbol': 'AMD',
 'exchangeName': 'NMS',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 322151400,
 'regularMarketTime': 1614978002,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 78.52,
 'chartPreviousClose': 2.37,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1614934800,
   'end': 1614954600,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1614954600,
   'end': 1614978000,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1614978000,
   'end': 1614992400,
   'gmtoffset': -18000}},
 'dataGranularity': '1d',
 'range': '5y',
 'validRanges': ['1d',
  '5d',
  '1mo',
  '3mo',
  '6mo',
  '1y',
  '2y',
  '5y',
  '10y',
  'ytd',
  'max']}

In [53]:
longMetadataAndPrice = metadata_price_action(symbol, longTimeInterval, timeRange)
longMetadata = metadataAndPrice["metadata"]
longMetadata

https://query1.finance.yahoo.com/v8/finance/chart/AMD?symbol=AMD&region=US&lang=en-US&includePrePost=false&interval=1wk&useYfid=true&range=5y&corsDomain=finance.yahoo.com&.tsrc=finance


{'currency': 'USD',
 'symbol': 'AMD',
 'exchangeName': 'NMS',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 322151400,
 'regularMarketTime': 1614978002,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 78.52,
 'chartPreviousClose': 2.37,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1614934800,
   'end': 1614954600,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1614954600,
   'end': 1614978000,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1614978000,
   'end': 1614992400,
   'gmtoffset': -18000}},
 'dataGranularity': '1d',
 'range': '5y',
 'validRanges': ['1d',
  '5d',
  '1mo',
  '3mo',
  '6mo',
  '1y',
  '2y',
  '5y',
  '10y',
  'ytd',
  'max']}

In [54]:
price = metadataAndPrice["price"]
# price.columns = ["timestamp", "Close", "Open", "High", "Volume", "Low"]
price = price.dropna()
price["Close"]=price["close"]
price["High"]=price["high"]
price["Low"]=price["low"]
price["Open"]=price["open"]

price

,timestamp,close,open,volume,high,low,Close,High,Low,Open
0,1457361000,2.470000,2.360000,8754300,2.470000,2.360000,2.470000,2.470000,2.360000,2.360000
1,1457447400,2.300000,2.440000,10026900,2.460000,2.300000,2.300000,2.460000,2.300000,2.440000
2,1457533800,2.260000,2.320000,13082400,2.350000,2.200000,2.260000,2.350000,2.200000,2.320000
3,1457620200,2.260000,2.290000,9603500,2.310000,2.150000,2.260000,2.310000,2.150000,2.290000
4,1457706600,2.520000,2.290000,21382400,2.540000,2.290000,2.520000,2.540000,2.290000,2.290000
...,...,...,...,...,...,...,...,...,...,...
1254,1614609000,86.389999,85.370003,32716700,86.500000,83.970001,86.389999,86.500000,83.970001,85.370003
1255,1614695400,84.129997,86.919998,34773100,86.949997,84.040001,84.129997,86.949997,84.040001,86.919998
1256,1614781800,80.860001,84.279999,43471600,84.379997,80.849998,80.860001,84.379997,80.849998,84.279999
1257,1614868200,77.750000,80.230003,60252900,81.809998,76.779999,77.750000,81.809998,76.779999,80.230003


In [55]:
longPrice = longMetadataAndPrice["price"]
# longPrice.columns = ["timestamp", "Close", "Open", "High", "Volume", "Low"]
longPrice

,timestamp,high,low,close,open,volume
0,1457326800,2.540000,2.150000,2.520000,2.360000,62849500
1,1457928000,2.930000,2.450000,2.930000,2.530000,123768600
2,1458532800,2.900000,2.620000,2.790000,2.900000,43988000
3,1459137600,2.980000,2.700000,2.830000,2.810000,53575000
4,1459742400,2.870000,2.610000,2.740000,2.830000,49734100
...,...,...,...,...,...,...
257,1612760400,94.220001,87.980003,93.769997,88.309998,201713800
258,1613365200,94.000000,87.309998,89.580002,93.900002,132524600
259,1613970000,88.300003,79.360001,84.510002,88.150002,231033500
260,1614574800,86.949997,76.779999,77.750000,85.370003,171214300


In [56]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from statistics import mean

def mySMA(myArr, period):
    return pd.Series(myArr).rolling(period).mean()
            
def myEMA(myArr, period):
    return pd.Series(myArr).ewm(adjust=False, span=period).mean()
    
def myMacd(myArr):
    twelve = myEMA(myArr, 8) 
    twentySix = myEMA(myArr, 26)
    macd = twelve-twentySix
    return macd

def myMacdSignal(myArr):
    return myEMA(myMacd(myArr), 9)

def myMacdHist(myArr):
    return myMacd(myArr) - myMacdSignal(myArr)
    
class SmaCross(Strategy):
    def init(self):
        tempprice = self.data.Close
        self.ma1 = self.I(myEMA, tempprice, 8)
        self.ma2 = self.I(myEMA, tempprice, 26)
        self.macd = self.I(myMacd, tempprice)
        self.macdSignal = self.I(myMacdSignal, tempprice)
        self.macdHist = self.I(myMacdHist, tempprice)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(price, SmaCross,
              exclusive_orders=True)
stats = bt.run()
bt.plot()

<ipython-input-56-e6145e1e6b27>:40: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(price, SmaCross,


In [9]:
basicStats = basic_stats(symbol)
basicStats

{'Previous Close': '621.44',
 'Open': '626.06',
 'Bid': '596.22 x 1400',
 'Ask': '596.50 x 3100',
 "Day's Range": '540.94 - 627.77',
 '52 Week Range': '70.10 - 900.40',
 'Volume': '89,395,616',
 'Avg. Volume': '40,189,206',
 'Market Cap': '573.945B',
 'Beta (5Y Monthly)': '2.06',
 'PE Ratio (TTM)': '934.30',
 'EPS (TTM)': '0.64',
 'Earnings Date': 'Apr 27, 2021 - May 03, 2021',
 'Forward Dividend & Yield': 'N/A (N/A)',
 'Ex-Dividend Date': 'N/A'}

In [9]:
extStats = extended_stock_stats(symbol)
extStats

{'Market Cap (intraday) 5': ['101.91B',
  '111.16B',
  '98.55B',
  '61.76B',
  '53.26B',
  '53.66B'],
 'Enterprise Value 3': ['100.15B',
  '109.97B',
  '97.67B',
  '61.08B',
  '52.48B',
  '53.56B'],
 'Trailing P/E ': ['40.84', '123.93', '157.67', '122.35', '151.60', '241.37'],
 'Forward P/E 1': ['44.25', '51.02', '49.26', '52.63', '39.84', '43.48'],
 'PEG Ratio (5 yr expected) 1': ['1.57',
  '1.13',
  '1.42',
  '1.55',
  '1.66',
  '1.55'],
 'Price/Sales (ttm)': ['10.40', '12.68', '12.67', '8.37', '7.57', '8.38'],
 'Price/Book (mrq)': ['17.46', '28.72', '29.82', '20.34', '18.84', '24.66'],
 'Enterprise Value/Revenue 3': ['10.26',
  '33.90',
  '34.87',
  '31.61',
  '29.39',
  '25.18'],
 'Enterprise Value/EBITDA 6': ['59.76',
  '167.63',
  '193.41',
  '238.59',
  '202.63',
  '180.94'],
 'Beta (5Y Monthly) ': '2.18',
 '52-Week Change 3': '60.01%',
 'S&P500 52-Week Change 3': '26.78%',
 '52 Week High 3': '99.23',
 '52 Week Low 3': '36.75',
 '50-Day Moving Average 3': '88.42',
 '200-Day Movi